In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import xlrd
import config

In [2]:
dbname = config.dbname
user = config.user
password = config.password
host = config.host
port = config.port

In [3]:
data_file_path = 'data/static_data/station_data.csv'
df = pd.read_csv(data_file_path)

In [4]:
def create_table(dbname, user, password, host, port, table_name):
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cur = conn.cursor()
    print("Check 2")
    # Create table if it doesn't exist
    cur.execute(sql.SQL("""
        CREATE TABLE IF NOT EXISTS {}
        (
            
            station_id INTEGER PRIMARY KEY NOT NULL,
            security INTEGER NOT NULL,
            operation_id INTEGER NOT NULL,
            stationType_id INTEGER NOT NULL,
            corporation_id INTEGER NOT NULL,
            solarSystem_id INTEGER NOT NULL,
            constellation_id INTEGER NOT NULL,
            region_id INTEGER NOT NULL,
            station_name VARCHAR(100) NOT NULL

        );
    """).format(sql.Identifier(table_name)))
    # Commit changes and close connection
    conn.commit()
    cur.close()
    conn.close()

In [5]:
rows = create_table(dbname, user, password, host, port, 'exchanges')

Check 2


In [6]:
def add_timepoint(dbname, user, password, host, port, data, table_name):
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cur = conn.cursor()
    # Insert data
    print(data.head())
    for _, row in data.iterrows():
        cur.execute(sql.SQL("""
            INSERT INTO {} (station_id, security, operation_id, stationType_id, corporation_id, solarSystem_id, constellation_id, region_id, station_name) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
        """).format(sql.Identifier(table_name)), 
            (row['stationID'], row['security'], row['operationID'], row['stationTypeID'], row['corporationID']
             , row['solarSystemID'], row['constellationID'], row['regionID'], row['stationName']))
    # Commit changes and close connection
    conn.commit()
    cur.close()
    conn.close()

In [7]:
add_timepoint(dbname, user, password, host, port, df, 'exchanges')


   stationID  security  operationID  stationTypeID  corporationID  \
0   60000004  0.708087           26           1531        1000002   
1   60000007  0.794424           26           1531        1000002   
2   60000010  0.807470           26           1531        1000002   
3   60000013  0.794424           26           1531        1000002   
4   60000016  0.830855           26           1531        1000002   

   solarSystemID  constellationID  regionID  \
0       30002780         20000407  10000033   
1       30002779         20000407  10000033   
2       30002776         20000407  10000033   
3       30002779         20000407  10000033   
4       30002778         20000407  10000033   

                                         stationName  
0    Muvolailen X - Moon 3 - CBD Corporation Storage  
1           Ono V - Moon 9 - CBD Corporation Storage  
2                 Annaro I - CBD Corporation Storage  
3           Ono V - Moon 2 - CBD Corporation Storage  
4  Tasabeshi VIII - Moon 13